# Goal 3: Calibration and Rectification

### Step #01: Load images 

In [37]:
import numpy as np
import cv2
import glob
import os
import datetime

In [38]:
# ---------------------------------------------------------------------------
# Constants
# ---------------------------------------------------------------------------
RELATIVE_RAW_DATASET_PATH = "../raw"
LEFT_IMG_PATH  = f"{RELATIVE_RAW_DATASET_PATH}/calib/image_02/data"
RIGHT_IMG_PATH = f"{RELATIVE_RAW_DATASET_PATH}/calib/image_03/data"

In [39]:
# Board definitions
boards = [
    {"name": "boardBig", "pattern_size": (11,7)},
    {"name": "boardMed", "pattern_size": (5,7)},
]

# VISUALIZATION SETTINGS
VISUALIZE = True
VIS_DELAY_MS = 0  # Wait 500ms between images. Set to 0 to wait for keypress.

# !!! CRITICAL !!! 
# Measure your real square size. 
# USE METERS to match KITTI format (e.g., 30mm = 0.03)
REAL_SQUARE_SIZE_M = 0.1

# Precompute object point templates
obj_templates = {}
for b in boards:
    nx, ny = b["pattern_size"]
    # We apply the real world scale here
    objp = np.zeros((nx*ny, 3), np.float32)
    objp[:, :2] = np.mgrid[0:nx, 0:ny].T.reshape(-1, 2) * REAL_SQUARE_SIZE_M
    obj_templates[b["name"]] = objp

In [ ]:
import cv2
import numpy as np
import glob
import os

# ---------------------------------------------------------------------------
# 1. CONFIGURATION
# ---------------------------------------------------------------------------
RELATIVE_RAW_DATASET_PATH = "../raw"
LEFT_IMG_PATH  = f"{RELATIVE_RAW_DATASET_PATH}/calib/image_02/data"

PROCESS_ALL_IMAGES = False  

# Visualization settings
DEBUG_VISUALIZE    = True 
VISUALIZE_DELAY    = 1

MIN_BOARD_WIDTH_PX = 25

# RMS IMPROVEMENT CONFIG
REJECTION_THRESH_PX = 0.5  # Throw away detections with error > 0.5 px
ENABLE_RATIONAL_MODEL = True # Set True if you have a very wide lens (adds k4, k5, k6)

# WARNING: Measure your physical boards! If one is printed at 95% scale, 
# your RMS will never go down.
boards = [
    {"name": "boardBig", "pattern_size": (11, 7), "square_size": 0.10},
    {"name": "boardMed", "pattern_size": (5, 7),  "square_size": 0.10}, 
]

# ---------------------------------------------------------------------------
# 2. DETECTION LOGIC
# ---------------------------------------------------------------------------

def get_image_files(directory):
    if not os.path.exists(directory):
        return []
    files = sorted(glob.glob(os.path.join(directory, "*.png")))
    if not files:
        files = sorted(glob.glob(os.path.join(directory, "*.jpg")))
    return files

def sharpen_image(gray_img):
    kernel = np.array([[0, -1, 0], [-1, 5, -1], [0, -1, 0]])
    return cv2.filter2D(gray_img, -1, kernel)

def detect_multiple_boards(img):
    gray_base = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
    gray_processing = sharpen_image(gray_base).copy()
    
    # IMPROVEMENT: Tighter criteria (100 iters or 0.0001 epsilon)
    subpix_criteria = (cv2.TERM_CRITERIA_EPS + cv2.TERM_CRITERIA_MAX_ITER, 100, 0.0001)
    
    detections = []
    
    # 255 = Searchable area, 0 = Masked (already found)
    search_mask = np.ones_like(gray_base, dtype=np.uint8) * 255

    for board in boards:
        name = board["name"]
        nx, ny = board["pattern_size"]
        sq_size = board["square_size"]
        
        max_instances = 15 
        count = 0
        
        while count < max_instances:
            # 1. Apply the mask
            masked_input = cv2.bitwise_and(gray_processing, gray_processing, mask=search_mask)

            flags = cv2.CALIB_CB_ADAPTIVE_THRESH | cv2.CALIB_CB_NORMALIZE_IMAGE 
            found, corners = cv2.findChessboardCorners(masked_input, (nx, ny), None)

            # 2. Upscaling attempt for small boards
            if not found:
                gray_up = cv2.resize(masked_input, None, fx=2.0, fy=2.0, interpolation=cv2.INTER_LINEAR)
                found_up, corners_up = cv2.findChessboardCorners(gray_up, (nx, ny), None)
                if found_up:
                    corners = (corners_up / 2.0).astype(np.float32)
                    found = True

            if not found:
                break 

            corners = corners.reshape(-1, 2).astype(np.float32)
            
            # 3. Size Check
            x,y,w,h = cv2.boundingRect(corners.astype(np.int32))
            if w < MIN_BOARD_WIDTH_PX or h < MIN_BOARD_WIDTH_PX:
                hull_pts = cv2.convexHull(corners.reshape(-1, 2).astype(np.int32))
                cv2.fillConvexPoly(search_mask, hull_pts, 0)
                continue

            # 4. IMPROVEMENT: Dynamic Subpixel Window Size
            # Calculate average distance between the first two corners
            dist = np.linalg.norm(corners[0] - corners[1])
            radius = int(dist / 2.5) 
            
            # Clamp constraints (min 3px, max 30px to prevent running away)
            radius = max(3, min(30, radius))
            win_size = (radius, radius)
            
            cv2.cornerSubPix(gray_base, corners, win_size, (-1, -1), subpix_criteria)

            detections.append({
                'name': name,
                'corners': corners.copy(),
                'pattern_size': (nx, ny),
                'square_size': sq_size,
                'debug_radius': radius # Storing for curiosity
            })

            # 5. Update Mask
            hull_pts = cv2.convexHull(corners.reshape(-1, 2).astype(np.int32))
            cv2.fillConvexPoly(search_mask, hull_pts, 0)
            search_mask = cv2.erode(search_mask, np.ones((10,10), np.uint8)) 
            
            count += 1

    return detections

def get_object_points(nx, ny, square_size):
    objp = np.zeros((ny * nx, 3), np.float32)
    objp[:, :2] = np.mgrid[0:nx, 0:ny].T.reshape(-1, 2)
    return objp * square_size

# ---------------------------------------------------------------------------
# 3. MAIN EXECUTION
# ---------------------------------------------------------------------------
if __name__ == "__main__":
    left_files = get_image_files(LEFT_IMG_PATH)

    objpoints_L = []
    imgpoints_L = []
    
    # Keeping track of which image produced which detection for debugging
    det_indices = [] 
    
    img_shape = None

    print(f"Found {len(left_files)} left images.")
    
    # --- A. GATHER DATA ---
    for i, f_left in enumerate(left_files):
        imgL = cv2.imread(f_left)
        if imgL is not None:
            if img_shape is None: img_shape = (imgL.shape[1], imgL.shape[0])
            
            detsL = detect_multiple_boards(imgL)
            
            for det in detsL:
                nx, ny = det['pattern_size']
                objpoints_L.append(get_object_points(nx, ny, det['square_size']))
                imgpoints_L.append(det['corners'])
                det_indices.append(i) # Track that this detection belongs to image 'i'

            print(f"[{i:02d}] {os.path.basename(f_left)} | Found {len(detsL)} boards")

            if DEBUG_VISUALIZE and len(detsL) > 0:
                vis = imgL.copy()
                for det in detsL:
                    color = (0, 255, 0) if det['name'] == "boardBig" else (0, 0, 255)
                    cv2.drawChessboardCorners(vis, det['pattern_size'], det['corners'], True)
                    cx, cy = np.mean(det['corners'], axis=0).ravel()
                    # Visualize the calculated subpixel window radius
                    text = f"{det['name']} r:{det['debug_radius']}"
                    cv2.putText(vis, text, (int(cx), int(cy)), cv2.FONT_HERSHEY_SIMPLEX, 0.5, color, 2)

                cv2.imshow("Debug Detection", cv2.resize(vis, None, fx=0.6, fy=0.6))
                cv2.waitKey(VISUALIZE_DELAY)
        else:
            print(f"[{i:02d}] Failed to read {f_left}")

        if not PROCESS_ALL_IMAGES: break

    if len(objpoints_L) == 0: 
        print("No detections found.")
        exit()

    # --- B. INITIAL CALIBRATION ---
    print("\n" + "="*60)
    print(f"  PHASE 1: INITIAL CALIBRATION ({len(objpoints_L)} detections)")
    print("="*60)

    calib_flags = cv2.CALIB_FIX_K3
    if ENABLE_RATIONAL_MODEL:
        calib_flags = cv2.CALIB_RATIONAL_MODEL

    retL, K1, D1, rvecs1, tvecs1 = cv2.calibrateCamera(
        objpoints_L, imgpoints_L, img_shape, None, None, flags=calib_flags
    )
    print(f"Initial RMS: {retL:.4f} px")

    # --- C. OUTLIER REJECTION ---
    print("\n" + "="*60)
    print("  PHASE 2: FILTERING OUTLIERS")
    print("="*60)

    objpoints_refined = []
    imgpoints_refined = []
    
    errors = []
    dropped_count = 0

    for i in range(len(objpoints_L)):
        # Project points back to image plane
        img_points_reproj, _ = cv2.projectPoints(objpoints_L[i], rvecs1[i], tvecs1[i], K1, D1)
        
        # FIX: Reshape projected points to match the detected points shape (N, 2)
        img_points_reproj = img_points_reproj.reshape(-1, 2)
        
        # Calculate Euclidean error
        error = cv2.norm(imgpoints_L[i], img_points_reproj, cv2.NORM_L2) / len(img_points_reproj)
        errors.append(error)

        if error < REJECTION_THRESH_PX:
            objpoints_refined.append(objpoints_L[i])
            imgpoints_refined.append(imgpoints_L[i])
        else:
            dropped_count += 1

    print(f"Original Avg Reprojection Error: {np.mean(errors):.4f} px")
    print(f"Dropped {dropped_count} detections with error > {REJECTION_THRESH_PX} px")
    print(f"Remaining detections: {len(objpoints_refined)}")
    
    if len(objpoints_refined) < 5:
        print("CRITICAL: Too many points dropped. Check if your boards are physically flat or measured correctly.")
        # Fallback to original if we lost too much data
        objpoints_refined = objpoints_L
        imgpoints_refined = imgpoints_L
    
    # --- D. FINAL CALIBRATION ---
    print(f"\n>>> Re-calibrating with refined dataset...")
    ret_final, K_final, D_final, rvecs_final, tvecs_final = cv2.calibrateCamera(
        objpoints_refined, imgpoints_refined, img_shape, None, None, flags=calib_flags
    )

    print(f"Final Refined RMS: {ret_final:.4f} px")
    print(f"K (Intrinsics):\n{np.array_str(K_final, precision=2, suppress_small=True)}")
    print(f"D (Distortion):\n{np.array_str(D_final, precision=3, suppress_small=True)}")

    # --- E. UNDISTORTION PREVIEW ---
    print("\n" + "="*60)
    print("  VISUALIZING RESULT")
    print("="*60)
    
    h, w = img_shape
    new_K, roi = cv2.getOptimalNewCameraMatrix(K_final, D_final, (w,h), 1, (w,h))

    for f_left in left_files:
        imgL = cv2.imread(f_left)
        if imgL is None: continue

        undistL = cv2.undistort(imgL, K_final, D_final, None, new_K)

        # Draw grid for visual reference
        step = 50
        for y in range(0, h, step):
            cv2.line(undistL, (0, y), (w, y), (0, 255, 0), 1)
            cv2.line(imgL, (0, y), (w, y), (0, 0, 255), 1) 

        combined = np.hstack((imgL, undistL))
        preview = cv2.resize(combined, None, fx=0.5, fy=0.5)
        cv2.putText(preview, f"RMS: {ret_final:.4f}", (50, 50), cv2.FONT_HERSHEY_SIMPLEX, 1, (0,255,0), 2)
        
        cv2.imshow("Left: Raw vs Undistorted", preview)
        
        key = cv2.waitKey(0)
        if key == 27: break # ESC

    cv2.destroyAllWindows()

Found 19 left images.
[00] 0000000000.png | Found 12 boards

  PHASE 1: INITIAL CALIBRATION (12 detections)
Initial RMS: 0.1974 px

  PHASE 2: FILTERING OUTLIERS
Original Avg Reprojection Error: 0.0305 px
Dropped 0 detections with error > 0.5 px
Remaining detections: 12

>>> Re-calibrating with refined dataset...
Final Refined RMS: 0.1974 px
K (Intrinsics):
[[1108.84    0.    692.86]
 [   0.   1093.94  255.46]
 [   0.      0.      1.  ]]
D (Distortion):
[[-0.095 -0.083 -0.002 -0.012  0.176  0.106  0.09  -0.1    0.     0.
   0.     0.     0.     0.   ]]

  VISUALIZING RESULT


KeyboardInterrupt: 